In [1]:
import pandas as pd
import numpy as np
from acquire import get_zillow_data

In [2]:
def fill_nulls(df):
    
    df.poolcnt = df.poolcnt.fillna(0)
    df.fireplacecnt = df.fireplacecnt.fillna(0)
    df.heatingorsystemdesc = df.heatingorsystemdesc.fillna('None')
    df.unitcnt = df.unitcnt.fillna(1)
    return df

In [3]:
def remove_outliers(df):
    #filter out bedrooms and bathrooms == 0
    df = df[(df.bedroomcnt > 0) & (df.bedroomcnt <= 7) & (df.bathroomcnt > 0) & (df.bathroomcnt <= 7)]
    #filter out houses less than 400 square feet
    df = df[(df.calculatedfinishedsquarefeet > 400) & (df.calculatedfinishedsquarefeet < 7000)]
    #filter out all units not equal to 1
    df = df[df.unitcnt == 1]
    return df

In [4]:
def create_features(df):
    df['age'] = 2017 - df.yearbuilt
    # create taxrate variable
    df['taxrate'] = df.taxamount/df.taxvaluedollarcnt
    # create acres variable
    df['acres'] = df.lotsizesquarefeet/43560
    # dollar per square foot-structure
    df['structure_dollar_per_sqft'] = df.structuretaxvaluedollarcnt/df.calculatedfinishedsquarefeet
    # dollar per square foot-land
    df['land_dollar_per_sqft'] = df.landtaxvaluedollarcnt/df.lotsizesquarefeet
    # ratio of beds to baths
    df['bed_bath_ratio'] = df.bedroomcnt/df.bathroomcnt
    #changing numbered labels into appropriate names
    df['county'] = df.fips.replace([6037, 6059, 6111],['los_angeles', 'orange', 'ventura'])
    #creating dummy variables
    county_df = pd.get_dummies(df.county)
    #adding dummies back into main dataframe
    df = pd.concat([df, county_df], axis=1)
    # 12447 is the ID for city of LA. 
    # I confirmed through sampling and plotting, as well as looking up a few addresses.
    df['cola'] = df['regionidcity'].apply(lambda x: 1 if x == 12447.0 else 0)
    #filter out outliers on new features
    df = df[(df.acres < 10) & (df.taxrate < .05)]
    #drop duplicate columns
    df = df.drop(columns = ['bathroomcnt', 'county', 'taxamount', 'taxvaluedollarcnt', 
                       'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 
                       'yearbuilt', 'lotsizesquarefeet'])
    return df

In [5]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    '''
    This function takes in a Dataframe, 
    proportion(0-1) of nulls required for a column
    and a proportion(0-1) of nulls required for rows
    then returns a dataframe without the nulls 
    under the threshold
    '''
    #setting threshold for row, only accepts integer
    thresh_row = int(round(prop_required_column*df.shape[0],0))
    #dropping nulls under threshold
    df.dropna(axis=1, thresh=thresh_row, inplace=True)
    #setting threshold for columns, only accepts integer
    thresh_col = int(round(prop_required_row*df.shape[1],0))
    #dropping nulls under threshold
    df.dropna(axis=0, thresh=thresh_col, inplace=True)
    return df 

In [6]:
df = get_zillow_data()

In [7]:
#filling nulls with appropriate values
df = fill_nulls(df)

In [8]:
df = remove_outliers(df)

In [9]:
df = create_features(df)

In [10]:
#filter out columns and rows with more than 40% null values
df = handle_missing_values(df, .6, .6)

In [11]:
#drop duplicate or unnecessary columns
df = df.drop(columns = ['propertylandusetypeid', 'calculatedbathnbr', 
                        'finishedsquarefeet12', 'heatingorsystemtypeid', 
                        'id', 'fips', 'fullbathcnt', 'propertyzoningdesc', 
                        'regionidcounty', 'id.1'])

In [12]:
def zillow_split(df):
    '''
    This function splits a dataframe into train, validate, and test sets
    '''
    train_and_validate, test = train_test_split(df, train_size=.8, random_state=123)
    train, validate = train_test_split(train_and_validate, train_size = .7, random_state=123)
    return train, validate, test

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train, validate, test = zillow_split(df)

In [15]:
cols_fixed = ['buildingqualitytypeid', 'regionidcity', 'censustractandblock', 'regionidzip', 'age']
for col in cols_fixed:
    mode = int(train[col].mode())
    train[col].fillna(value = mode, inplace = True)
    validate[col].fillna(value = mode, inplace = True)
    test[col].fillna(value = mode, inplace = True)


/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [16]:
#missing continuous values will be replaced with the median
cols_cont = ['taxrate', 'acres', 'structure_dollar_per_sqft', 'land_dollar_per_sqft']
for col in cols_cont:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28854 entries, 3961 to 42104
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      28854 non-null  int64  
 1   bedroomcnt                    28854 non-null  float64
 2   buildingqualitytypeid         28854 non-null  float64
 3   calculatedfinishedsquarefeet  28854 non-null  float64
 4   fireplacecnt                  28854 non-null  float64
 5   latitude                      28854 non-null  float64
 6   longitude                     28854 non-null  float64
 7   poolcnt                       28854 non-null  float64
 8   propertycountylandusecode     28854 non-null  object 
 9   rawcensustractandblock        28854 non-null  float64
 10  regionidcity                  28854 non-null  float64
 11  regionidzip                   28854 non-null  float64
 12  roomcnt                       28854 non-null  float64
 13

In [18]:
validate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12367 entries, 7806 to 29620
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      12367 non-null  int64  
 1   bedroomcnt                    12367 non-null  float64
 2   buildingqualitytypeid         12367 non-null  float64
 3   calculatedfinishedsquarefeet  12367 non-null  float64
 4   fireplacecnt                  12367 non-null  float64
 5   latitude                      12367 non-null  float64
 6   longitude                     12367 non-null  float64
 7   poolcnt                       12367 non-null  float64
 8   propertycountylandusecode     12367 non-null  object 
 9   rawcensustractandblock        12367 non-null  float64
 10  regionidcity                  12367 non-null  float64
 11  regionidzip                   12367 non-null  float64
 12  roomcnt                       12367 non-null  float64
 13

In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10306 entries, 39813 to 16358
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      10306 non-null  int64  
 1   bedroomcnt                    10306 non-null  float64
 2   buildingqualitytypeid         10306 non-null  float64
 3   calculatedfinishedsquarefeet  10306 non-null  float64
 4   fireplacecnt                  10306 non-null  float64
 5   latitude                      10306 non-null  float64
 6   longitude                     10306 non-null  float64
 7   poolcnt                       10306 non-null  float64
 8   propertycountylandusecode     10306 non-null  object 
 9   rawcensustractandblock        10306 non-null  float64
 10  regionidcity                  10306 non-null  float64
 11  regionidzip                   10306 non-null  float64
 12  roomcnt                       10306 non-null  float64
 1